In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os


In [2]:
a = pd.read_excel('/Users/harshvardhangupta/Documents/NLP/Credit Modeling/case_study1.xlsx')

In [3]:
b = pd.read_excel('Credit Modeling/case_study2.xlsx')

In [11]:
# a.shape
# b.shape

In [12]:
df1 = a.copy()
df2 = b.copy()

In [13]:
# df1.loc[df1['Age_Oldest_TL'] != -99999 ]

## Observations

- df1 : 26 columns and 51336 rows
- df2 : 62 columns and 51336 rows
### Processing
#### Df1
- removed 40 null rows

#### Df2
- removed columns, if having more than 10000 null values
    - 8 such columns were removed
- removed rows , if having less than 10000 null values
    - 9270 rows removed, which is about 18% of original rows
##### Df1 and Df2 merged (inner join) to form Df

In [14]:
# Remove nulls
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

In [15]:
df1.shape # 40 rows removed

(51296, 26)

In [8]:
# type(df1['Age_Oldest_TL'] != -99999)

In [9]:
# df2.loc[df2['time_since_recent_payment'] == -99999].shape

# type(df2.loc[df2['time_since_recent_payment'] == -99999])


In [10]:
# df2.columns    #index object
# df2.columns.values        #array
# df2.columns.values.tolist()      #list


# len(df2.columns.values.tolist())

In [16]:
# removing columns where null values > 10000

columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed.append(i)

In [17]:
len(columns_to_be_removed)

8

In [18]:
df2 = df2.drop(columns_to_be_removed, axis =1)

In [19]:
print('df1 shape : ', df1.shape)
print('df2 shape : ', df2.shape)

df1 shape :  (51296, 26)
df2 shape :  (51336, 54)


In [20]:
# removing rows where null values < 10000
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]

In [21]:
print('df1 shape : ', df1.shape)
print('df2 shape : ', df2.shape)

df1 shape :  (51296, 26)
df2 shape :  (42066, 54)


In [17]:
# df2.isna().sum()   #no null values now
#df1.isna().sum()    #no null values now

In [22]:
# Checking common column names
for i in df1.columns:
    if i in df2.columns:
        print (i)

PROSPECTID


In [23]:
# Merge the two dataframes, inner join so that no nulls are present
df = pd. merge ( df1, df2, how ='inner', on='PROSPECTID')


In [24]:
print('df1 shape : ', df1.shape)
print('df2 shape : ', df2.shape)
print('df shape : ', df.shape)

df1 shape :  (51296, 26)
df2 shape :  (42066, 54)
df shape :  (42064, 79)


In [32]:
df.sample(3)


,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
31761,38774,35,28,7,4,3,0.114,0.086,0.2,0.8,...,0.0,0.0,0.0,0.0,1,0,others,others,746,P1
35468,43309,5,4,1,0,0,0.000,0.000,0.2,0.8,...,0.0,0.0,0.0,0.0,0,0,others,others,700,P2
21883,26745,2,1,1,1,1,0.500,0.500,0.5,0.5,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,others,676,P2


In [33]:
# check how many columns are categorical
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [34]:
print(df['MARITALSTATUS'].value_counts())
print('\n')
print(df['last_prod_enq2'].value_counts())

Married    30886
Single     11178
Name: MARITALSTATUS, dtype: int64


ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: last_prod_enq2, dtype: int64


In [35]:
df['EDUCATION'].value_counts()

GRADUATE          14140
12TH              11703
SSC                7241
UNDER GRADUATE     4572
OTHERS             2291
POST-GRADUATE      1898
PROFESSIONAL        219
Name: EDUCATION, dtype: int64

In [36]:
df['first_prod_enq2'].value_counts()

others          20640
ConsumerLoan    11075
PL               4431
AL               2641
CC               1988
HL               1289
Name: first_prod_enq2, dtype: int64

## More Observations

#### For Categorical Variables
- Since all five categorical features have pval <=0.05, we will accept all

#### For Numerical Variables
- total numeric columns = 72
##### after removing columns with high multicollinearity
- we removed 33 columns
- therefore we have vif_data with 39 columns and 42064 rows

##### after removing columns that were found Not significant (aplha = 0.05) in ANOVA test
- we removed 2 columns
- therefore we are remaining with 37 columns

In [37]:
# Chi-square test
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', pval)

MARITALSTATUS --- 3.5781808610388605e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.9079361001865664e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.849976105554191e-287


In [38]:
# creating list of numerical columns
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)
        

In [28]:
# len(numeric_columns)  #72
# numeric_columns

In [29]:
# vif_data = df[numeric_columns]
# vif_data

In [39]:
#VIF sequentially check

vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

for i in range (0,total_columns):
    
    vif_value = variance_inflation_factor(vif_data, column_index)
    print ("Iteration no.", i ,'---', f"Column_index - {column_index}" , '---',vif_value)
    
    
    if vif_value <= 6:
        columns_to_be_kept.append( numeric_columns[i] )
        column_index = column_index+1
    
    else:
        vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)   # after dropping, the next column will become the 0th column

/Users/harshvardhangupta/.local/share/virtualenvs/NLP-4NjDqJgX/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Iteration no. 0 --- Column_index - 0 --- inf


/Users/harshvardhangupta/.local/share/virtualenvs/NLP-4NjDqJgX/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Iteration no. 1 --- Column_index - 0 --- inf
Iteration no. 2 --- Column_index - 0 --- 11.320180023967982
Iteration no. 3 --- Column_index - 0 --- 8.36369803500036
Iteration no. 4 --- Column_index - 0 --- 6.5206478777909425
Iteration no. 5 --- Column_index - 0 --- 5.149501618212613
Iteration no. 6 --- Column_index - 1 --- 2.6111110405797335


/Users/harshvardhangupta/.local/share/virtualenvs/NLP-4NjDqJgX/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Iteration no. 7 --- Column_index - 2 --- inf
Iteration no. 8 --- Column_index - 2 --- 1788.7926256209232
Iteration no. 9 --- Column_index - 2 --- 8.601028256477212
Iteration no. 10 --- Column_index - 2 --- 3.832800792153082
Iteration no. 11 --- Column_index - 3 --- 6.0996533816466405
Iteration no. 12 --- Column_index - 3 --- 5.581352009642814
Iteration no. 13 --- Column_index - 4 --- 1.9855843530987702


/Users/harshvardhangupta/.local/share/virtualenvs/NLP-4NjDqJgX/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Iteration no. 14 --- Column_index - 5 --- inf
Iteration no. 15 --- Column_index - 5 --- 4.809538302819332
Iteration no. 16 --- Column_index - 6 --- 23.270628983464636
Iteration no. 17 --- Column_index - 6 --- 30.595522588099946
Iteration no. 18 --- Column_index - 6 --- 4.384346405965575
Iteration no. 19 --- Column_index - 7 --- 3.064658415523413
Iteration no. 20 --- Column_index - 8 --- 2.898639771299223
Iteration no. 21 --- Column_index - 9 --- 4.377876915347339
Iteration no. 22 --- Column_index - 10 --- 2.2078535836958477
Iteration no. 23 --- Column_index - 11 --- 4.916914200506877
Iteration no. 24 --- Column_index - 12 --- 5.21470203006474
Iteration no. 25 --- Column_index - 13 --- 3.3861625024231503
Iteration no. 26 --- Column_index - 14 --- 7.84058330947899
Iteration no. 27 --- Column_index - 14 --- 5.255034641721459


/Users/harshvardhangupta/.local/share/virtualenvs/NLP-4NjDqJgX/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Iteration no. 28 --- Column_index - 15 --- inf
Iteration no. 29 --- Column_index - 15 --- 7.380634506427207
Iteration no. 30 --- Column_index - 15 --- 1.4210050015175721
Iteration no. 31 --- Column_index - 16 --- 8.083255010190301
Iteration no. 32 --- Column_index - 16 --- 1.6241227524040012
Iteration no. 33 --- Column_index - 17 --- 7.257811920140015
Iteration no. 34 --- Column_index - 17 --- 15.596243832683006
Iteration no. 35 --- Column_index - 17 --- 1.825857047132431
Iteration no. 36 --- Column_index - 18 --- 1.508083945003272
Iteration no. 37 --- Column_index - 19 --- 2.172088834824582
Iteration no. 38 --- Column_index - 20 --- 2.6233975535272367
Iteration no. 39 --- Column_index - 21 --- 2.2959970812106216
Iteration no. 40 --- Column_index - 22 --- 7.360578319196457
Iteration no. 41 --- Column_index - 22 --- 2.1602387773102514
Iteration no. 42 --- Column_index - 23 --- 2.8686288267891493
Iteration no. 43 --- Column_index - 24 --- 6.458218003637239
Iteration no. 44 --- Column_ind

In [40]:
vif_data.shape

(42064, 39)

In [41]:
# check Anova for columns_to_be_kept 

from scipy.stats import f_oneway

In [42]:
# len(columns_to_be_kept)
columns_to_be_kept

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'pct_currentBal_all_TL',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag']

In [43]:
# list(df['pct_tl_open_L6M'])
gt = list(df['pct_tl_open_L6M'])
pk = list(df['Approved_Flag'])

# for value, group in zip(gt , pk):
#     print(value, '---', group)

In [44]:
columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])  
    b = list(df['Approved_Flag'])  
    
    group_P1 = [value for value, group in zip(a, b) if group == 'P1']  # it stores all the values of a column i which belong to group P1
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']  # it stores all the values of a column i which belong to group P2
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

In [45]:
# columns_to_be_kept_numerical
len(columns_to_be_kept_numerical)

37

In [46]:
#columns which were removed after ANOVA
for i in columns_to_be_kept:
    if i not in columns_to_be_kept_numerical:
        print(i)

num_lss_12mts
pct_currentBal_all_TL


In [47]:
# feature selection is done for cat and num features

# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]


In [48]:
df.shape

(42064, 43)

In [40]:
# new_df = df.copy()
# new_df.head(8)
# df_try = new_df.copy()
# df_try.shape

## Latest Observations

- new df after feature selection is complete
- 37 numerical + 5 categorical + 1 response variable
- total 43 columns
- 42064 rows
- all the columns are associated with the reponse variable.
- multicollinearity has been addressed.

### Encoding Categorical Variables

- Education : ordinal encoding  
- Rest all : one hot encoding
- After Encoding - 42064 rows and 55 columns
    
**NOTE - label encoding is for target variables and not for predictor variables**
  

In [49]:
# Label encoding for the categorical features
['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [42]:

# df['MARITALSTATUS'].unique()    
# df['EDUCATION'].unique()
# df['GENDER'].unique()
# df['last_prod_enq2'].unique()
# df['first_prod_enq2'].unique()


In [43]:
# from sklearn.preprocessing import OrdinalEncoder

In [44]:
# oe = OrdinalEncoder(categories=['OTHERS','SSC','12TH','PROFESSIONAL', 'UNDER GRADUATE', 'POST-GRADUATE' ])

In [50]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 2
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 3
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 6
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 5
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 7
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 4

In [68]:
df['EDUCATION'].value_counts()
# df['EDUCATION'] = df['EDUCATION'].astype(int)
# df.info()

6    14140
3    11703
2     7241
5     4572
1     2291
7     1898
4      219
Name: EDUCATION, dtype: int64

In [69]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])


In [80]:
# df_encoded.info()
df_encoded['EDUCATION']= df_encoded['EDUCATION'].astype('int')

In [81]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [49]:
# from sklearn.preprocessing import OrdinalEncoder

In [50]:
# oe = OrdinalEncoder(categories=[['OTHERS','SSC','12TH','PROFESSIONAL', 'UNDER GRADUATE', 'GRADUATE', 'POST-GRADUATE']])

In [51]:
# df_try.shape

In [52]:
# type(df_try['EDUCATION'])
# df_try[['EDUCATION']]

In [53]:
# oe.fit(df_try[['EDUCATION']])

In [54]:
# oe.transform(df_try[['EDUCATION']])

In [53]:
k = df_encoded.describe()
# type(k)

## Model Observations


### Random Forest


### XGBoost


### Decision Tree

In [82]:
# Random Forest

y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )



In [83]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [84]:
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [85]:
y_pred = rf_classifier.predict(x_test)

In [86]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()




Accuracy: 0.7658385831451325

Class p1:
Precision: 0.8339181286549707
Recall: 0.703155818540434
F1 Score: 0.7629748528624933

Class p2:
Precision: 0.7970398094589997
Recall: 0.9286422200198216
F1 Score: 0.8578229424150874

Class p3:
Precision: 0.44976816074188564
Recall: 0.21962264150943397
F1 Score: 0.295131845841785

Class p4:
Precision: 0.7299128751210068
Recall: 0.7327502429543246
F1 Score: 0.7313288069835111



In [87]:
from sklearn.metrics import classification_report , confusion_matrix

In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          P1       0.83      0.70      0.76      1014
          P2       0.80      0.93      0.86      5045
          P3       0.45      0.22      0.30      1325
          P4       0.73      0.73      0.73      1029

    accuracy                           0.77      8413
   macro avg       0.70      0.65      0.66      8413
weighted avg       0.74      0.77      0.74      8413



In [89]:
print(confusion_matrix(y_test, y_pred))

[[ 713  301    0    0]
 [ 112 4685  195   53]
 [  29  779  291  226]
 [   1  113  161  754]]


In [71]:
# x.info()

In [90]:
# XGBoost 

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [91]:
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'], axis = 1 )

In [92]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # y_encoded is returned as an array

In [93]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


In [94]:
xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

In [95]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.7734458576013312

Class p1:
Precision: 0.8211991434689507
Recall: 0.7564102564102564
F1 Score: 0.7874743326488707

Class p2:
Precision: 0.821920257464688
Recall: 0.911199207135778
F1 Score: 0.86426019928558

Class p3:
Precision: 0.45756880733944955
Recall: 0.3011320754716981
F1 Score: 0.3632225762403277

Class p4:
Precision: 0.7337278106508875
Recall: 0.7230320699708455
F1 Score: 0.7283406754772394



In [96]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

In [97]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

In [98]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.71

Class p1:
Precision: 0.722057368941642
Recall: 0.7199211045364892
F1 Score: 0.7209876543209877

Class p2:
Precision: 0.8093664982510688
Recall: 0.8255698711595639
F1 Score: 0.8173878912766166

Class p3:
Precision: 0.34335443037974683
Recall: 0.32754716981132076
F1 Score: 0.33526458091927386

Class p4:
Precision: 0.6522177419354839
Recall: 0.6287657920310982
F1 Score: 0.6402770905492331



## Hyperparameter Tuning

In [99]:
from sklearn.model_selection import GridSearchCV

In [100]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


In [101]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

In [102]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=4,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [ ]:
print("Best Hyperparameters:", grid_search.best_params_)